#  Some Installations First 

In [ ]:
%%capture
!pip install mediapipe

#  Get Data from URL 

In [ ]:
!wget http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/Database/AFLW2000-3D.zip
!unzip AFLW2000-3D.zip

# Required Libraries 

In [ ]:
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
import dlib
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
from google.colab.patches import cv2_imshow #when using colab
import glob
from pathlib import Path
from PIL import Image, ImageFilter

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error ,r2_score

import warnings
warnings.filterwarnings('ignore')

# Load Data 

In [ ]:
mylist = [Path(f).stem for f in glob.glob("AFLW2000/*.mat")]
sio.loadmat(os.path.join('AFLW2000/',mylist[0]))

In [ ]:
def get_list_from_filenames(file_path):
    # input:    relative path to .txt file with file names
    # output:   list of relative path names
    lines = [Path(f).stem for f in glob.glob(file_path)]
    return lines

In [ ]:
images = get_list_from_filenames("AFLW2000/*.jpg")
images

## 1. extract pitch , yaw and roll from matlab file

In [ ]:
def get_ypr_from_mat(mat_path):
    # Get yaw, pitch, roll from .mat annotation.
    # They are in radians
    mat = sio.loadmat(mat_path)
    # [pitch yaw roll tdx tdy tdz scale_factor]
    pre_pose_params = mat['Pose_Para'][0]
    # Get [pitch, yaw, roll]
    pose_params = pre_pose_params[:3]
    return pose_params

## 2. extract data points from each image 

In [ ]:
def get_imageData (r):
  random_file = r
  list_x = []
  list_y = []
  result = pd.DataFrame()
  xs = ['x'+str(i) for i in range(1,469)]
  ys = ['y'+str(i) for i in range(1,469)]
  labels = ['pitch', 'yaw', 'roll']
  column_names = xs + ys + labels


    
  faceModule = mediapipe.solutions.face_mesh
      # loading image and its correspinding mat file
  with faceModule.FaceMesh(static_image_mode=True,max_num_faces=1) as faces:
          # loading the image
          image = cv2.imread('/content/AFLW2000/'+random_file+'.jpg')
          # processing the face to extract the landmark points (468 point) for each x,y,z
          results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
          if results.multi_face_landmarks != None: 
            # looping over the faces in the image
            for face in results.multi_face_landmarks:
                for landmark in face.landmark:
                    x = landmark.x
                    y = landmark.y
                    shape = image.shape 
                    relative_x = int(x * shape[1])
                    list_x = np.append(list_x,int(relative_x))
                    relative_y = int(y * shape[0])
                    list_y = np.append(list_y,relative_y) 
                    
                    x = pd.DataFrame(list_x.reshape(1,-1))
                    y = pd.DataFrame(list_y.reshape(1,-1))
                    xy = pd.DataFrame(pd.concat([x, y], join='inner', axis=1))
                    pyr=pd.DataFrame(get_ypr_from_mat('AFLW2000/' + random_file + '.mat').reshape(1,-1))
                    result = pd.DataFrame(pd.concat([xy, pyr], join='inner', axis=1))
  
  if len(list_x) != 0 :
    result.columns = column_names  
    return len(list_x),result
  else : 
    return 0 ,0
    

## 3. data preprocessing and normalization

In [ ]:
images = get_list_from_filenames("AFLW2000/*.jpg")
final = pd.DataFrame ()


for img in images :
    length,d = get_imageData (img)
    if length != 0 :
          final = final.append(d, ignore_index=True)
    else :
          idx = images.index(img)
  
final.to_csv('data.csv',index=False)


***NOTE* : It takes too long to extract data from images , so the csv file "data_img" cantains the dataframe of all the features and all labels .**

In [ ]:
data = pd.read_csv('data_img.csv')

X = data.iloc[:,:-3]
pitch =pd.DataFrame(data.iloc[:,-3])
yaw =pd.DataFrame(data.iloc[:,-2])
roll =pd.DataFrame(data.iloc[:,-1])



In [ ]:
df = X.copy()

x_norm = X.loc[:,'x2']
y_norm = X.loc[:,'y2']


In [ ]:
import re
for i in df.columns:
  if re.search('x', i):
    df[i] = df[i] - x_norm    
  if re.search('y', i):
    df[i] = df[i] - y_norm





In [ ]:

ratio = ((df.x10 ** 2 ) +   (df.y10 ** 2 ) ** 0.5)
ratio
normalized_df = df.copy()
normalized_df.shape
for i in normalized_df.columns:
    normalized_df[i] = normalized_df[i] / ratio  

normalized_df.head()
normalized_df.to_csv('normal_data.csv',index=False)

In [ ]:
normalized_df = pd.read_csv('normal_data.csv')
normalized_df.shape

##  4. train 3 different SVR to predict pitch ,yaw , roll 

In [ ]:
from sklearn.preprocessing import StandardScaler
#train_test_spilt
Xp_train, Xp_test, p_train, p_test = train_test_split(normalized_df, pitch, test_size=0.2, random_state=42)
# pitch prediction
scaler = StandardScaler()
Xp_scale=scaler.fit_transform(Xp_train)
p_reg =SVR(C=10, degree = 4)
p_reg.fit(Xp_scale, p_train)
p_pred = pd.DataFrame(p_reg.predict (Xp_test))
p_pred.columns = ["pitch"]
p_pred
#########################################
#train 
msP = mean_squared_error(p_train,pd.DataFrame(p_reg.predict (Xp_train)))
print("MSE of Training : %.2f" %msP)
rP = r2_score( p_train,pd.DataFrame(p_reg.predict (Xp_train)) )
print("R2 score of Training : %.2f" %rP)
#MSE 
mseP = mean_squared_error(p_test,p_pred)
print("MSE of pitch : %.2f" %mseP)
r2P = r2_score(p_test,p_pred)
print("R2 score of pitch : %.2f" %r2P)



# Note : r2-score is not effictive when it is not linear Regression 

In [ ]:
#train_test_spilt
Xy_train, Xy_test, y_train, y_test = train_test_split(normalized_df, yaw, test_size=0.33, random_state=42)
# yaw prediction
y_reg =SVR(C=1.0, epsilon=0.2)
y_reg.fit(Xy_train, y_train)
y_pred = pd.DataFrame(y_reg.predict (Xy_test))
y_pred.columns = ["yaw"]
y_pred
####################################
#train
msY =mean_squared_error(y_train,pd.DataFrame(y_reg.predict (Xy_train)))
print("MSE of Training  : %.2f" %msY)
rY =r2_score(y_train,pd.DataFrame(y_reg.predict (Xy_train)))
print("R2 score of Training : %.2f" % rY)
#MSE 
mseY =mean_squared_error(y_test,y_pred)
print("MSE of yaw  : %.2f" %mseY)
r2Y =r2_score(y_test,y_pred)
print("R2 score of yaw : %.2f" % r2Y)

In [ ]:
#train_test_spilt
Xr_train, Xr_test, r_train, r_test = train_test_split(normalized_df, roll, test_size=0.33, random_state=42)
# roll prediction
r_reg =SVR(C=1.0, epsilon=0.2,degree = 5)
r_reg.fit(Xr_train, r_train)
r_pred = pd.DataFrame(r_reg.predict (Xr_test))
r_pred.columns = ["roll"]
r_pred
#################################
#train
msR =mean_squared_error(r_train,pd.DataFrame(r_reg.predict (Xr_train)))
print("MSE of Training  : %.2f" %msR)
rR =r2_score(r_train,pd.DataFrame(r_reg.predict (Xr_train)))
print("R2 score of Training : %.2f" % rR)
#MSE 
mseR =mean_squared_error(r_test,r_pred)
print("MSE of roll  : %.2f" %mseR)
r2R =r2_score(r_test,r_pred)
print("R2 score of roll : %.2f" % r2R)

## 5. prepare test data from frames of video 

In [ ]:
def get_testData (r):
  
  list_x = []
  list_y = []
  result = pd.DataFrame()
    
  xs = ['x'+str(i) for i in range(1,469)]
  ys = ['y'+str(i) for i in range(1,469)]
  column_names = xs + ys 


    
  faceModule = mediapipe.solutions.face_mesh
      # loading image and its correspinding mat file
  with faceModule.FaceMesh(static_image_mode=True,max_num_faces=1) as faces:
          # loading the image
          image = r
          
          # processing the face to extract the landmark points (468 point) for each x,y,z
          results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
          
          if results.multi_face_landmarks != None: 
            # looping over the faces in the image
            for face in results.multi_face_landmarks:
                for landmark in face.landmark:
                    x = landmark.x
                    y = landmark.y
                    shape = image.shape 
                    relative_x = int(x * shape[1])
                    list_x = np.append(list_x,int(relative_x))
                    relative_y = int(y * shape[0])
                    list_y = np.append(list_y,relative_y) 
                   
                    x = pd.DataFrame(list_x.reshape(1,-1))
                    y = pd.DataFrame(list_y.reshape(1,-1))
                    xy = pd.DataFrame(pd.concat([x, y], join='inner', axis=1))
                    #cv2.putText(image, str(relative_y), (int(relative_x),int(relative_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 2)
                    #cv2.circle(image, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)
 
  if len(list_x) != 0 :
    xy.columns = column_names  
    return len(list_x),xy
  else : 
    return 0 ,0

In [ ]:

def prepare_frame (frame):

        t = pd.DataFrame ()
        

        nx,d = get_testData (frame)
        if nx != 0 :
          t = t.append(d, ignore_index=True)
        else :
          output =  frame 
        t.to_csv('test_data.csv',index=False)
        test = pd.read_csv ('test_data.csv')
        arr = np.array(test)
        x_origin = arr[0][1]
        y_origin = arr[0][469]



        import re
        for idx in test.columns:
          if re.search('x', idx):
            test[idx] = test[idx] - x_origin    
          if re.search('y', idx):
            test[idx] = test[idx] - y_origin

        t_ratio = ((test.x10 ** 2 ) +   (test.y10 ** 2 ) ** 0.5)

        normalized_test= test.copy()
      
        for i in normalized_test.columns:
            normalized_test[i] = normalized_test[i] / ratio  
     
        p1=p_reg.predict (normalized_test)
        y1=y_reg.predict (normalized_test)
        r1=r_reg.predict (normalized_test)
      
        return p1 , y1 , r1 , x_origin ,  y_origin
 

## 6. prepare and smoth frames with predicted pitch , yaw , roll 

In [ ]:
def smothing (frame):
  p_list = []
  y_list = []
  r_list = []
  for i in range (5):
    p,y,r,xx,yy = prepare_frame (frame)
    p_list.append (p)
    y_list.append (y)
    r_list.append (r)
   
  avg_p = sum(p_list) / len (p_list) 
  avg_y = sum(y_list) / len (y_list) 
  avg_r = sum(r_list) / len (r_list) 

  return avg_p ,avg_y,avg_r,xx,yy

In [ ]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2
   
    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [ ]:
#construct video from frames


cap = cv2.VideoCapture("/content/noor.mp4")


# Get video metadata
video_fps = cap.get(cv2.CAP_PROP_FPS),
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)


out = cv2.VideoWriter('/content/final_isa.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 30,  frameSize=(int(width), int(height)))
while True:
    ret, frame = cap.read()
    if not ret: break 
   
    #f = prepare_frame (frame)
    p1,y1,r1,x_origin,y_origin =smothing (frame)
    f = draw_axis(frame,p1,y1,r1,x_origin,y_origin)
    out.write(f) # write frame

    
    
        
# release and destroy windows
out.release()
cap.release()
cv2.destroyAllWindows()

In [ ]:
#test video
import ipywidgets as widgets
gif_file = "noor.gif"
display(widgets.HTML(f'<img src="{gif_file}" width="750" align="center">'))

In [ ]:
#display the created vedio 
cap = cv2.VideoCapture("/content/final_isa.avi")
if (cap.isOpened()== False):
  print("Error opening video stream or file")


# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:

    # Display the resulting frame
    cv2_imshow(frame)

    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

In [ ]:
#test video with Directions 
import ipywidgets as widgets
gif_file = "final_isa.gif"
display(widgets.HTML(f'<img src="{gif_file}" width="750" align="center">'))